### Passos necessários

1. Ler as imagens digitais;
2. Remover os background;
3. criar os embeddings;
4. Salvar os embeddings em um vector store ou refazer sempre que for executar;
<!-- 3. Pegar áreas novas, criar embeddings e classificar
4. Salva junto uma classe no vector store, e retorná-lo -->

In [ ]:
# Ambiente vit conda
# conda create -n vit python==3.10
# conda activate vit

In [ ]:
#!pip install faiss-gpu
#!pip install faiss-cpu
#!pip install -r requirements.txt

In [ ]:
# Lendo as bibliotecas
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import os
import torch

from rembg import remove
import io


import pandas as pd
import numpy as np

import faiss

In [ ]:
# Pre-processar as imagens -
# Padronizar as imagens e remover background

def processar_imagens_otimizado(caminho_pasta_entrada, caminho_pasta_saida, largura=640, altura=480):
    # (Mesma função processar_imagens_otimizado que você já tem)
    if not os.path.exists(caminho_pasta_saida):
        os.makedirs(caminho_pasta_saida)
        print(f"Pasta de saída criada: {caminho_pasta_saida}")

    lista_indices_caminhos = []
    indice_atual = 0

    for raiz, _, arquivos in os.walk(caminho_pasta_entrada):
        arquivos.sort() # Garante uma ordem consistente
        
        for arquivo in arquivos:
            if arquivo.lower().endswith(('.png', '.jpg', '.jpeg', '.webp', '.bmp', '.tiff')):
                caminho_completo_entrada = os.path.join(raiz, arquivo)
                
                lista_indices_caminhos.append({
                    'indice': indice_atual,
                    'caminho_original': caminho_completo_entrada
                })
                indice_atual += 1

                nome_arquivo_sem_ext, extensao = os.path.splitext(arquivo)
                extensao_saida = '.png' if extensao.lower() != '.png' else extensao
                nome_arquivo_saida = f"{nome_arquivo_sem_ext}_processado{extensao_saida}"
                caminho_completo_saida = os.path.join(caminho_pasta_saida, nome_arquivo_saida)

                try:
                    img_original = Image.open(caminho_completo_entrada)
                    img_redimensionada = img_original.resize((largura, altura), Image.LANCZOS)

                    buffer_img = io.BytesIO()
                    img_redimensionada.save(buffer_img, format='PNG' if extensao_saida == '.png' else img_original.format)
                    buffer_img.seek(0)

                    bytes_entrada = buffer_img.read()
                    bytes_saida = remove(bytes_entrada)

                    with open(caminho_completo_saida, 'wb') as o:
                        o.write(bytes_saida)
                    print(f"Processado e salvo: {caminho_completo_saida} (Original: {caminho_completo_entrada})")

                except FileNotFoundError:
                    print(f"Erro: Arquivo não encontrado: {caminho_completo_entrada}")
                except Exception as e:
                    print(f"Erro ao processar {caminho_completo_entrada}: {e}")

    return lista_indices_caminhos


In [ ]:
# --- Exemplo de uso ---
# Padronizar as imagens e remover background
pasta_entrada = 'database/natural_images'
pasta_saida = 'database/natural_images_without_bg'
arquivo_csv_saida = 'database/indices_imagens_preprocessadas.csv' # ALtere este caminho

# Processa as imagens e obtém a lista de índices/caminhos
info_imagens = processar_imagens_otimizado(pasta_entrada, pasta_saida,largura=224, altura=224)

In [ ]:
# Salvando em CSV a lista de arquivos originais
df = pd.DataFrame(info_imagens)
df.to_csv(arquivo_csv_saida, index=False, encoding='utf-8') # index=False para não salvar o índice do DataFrame

In [ ]:
# Ler as imagens da pasta database/natural_images_without_bg
def ler_imagens(lista_imagens, raiz):
    imagens = []

    for arquivo in lista_imagens:
        nome_arquivo = os.path.basename(arquivo)
        nome_arquivo_sem_ext, extensao = os.path.splitext(nome_arquivo)
        extensao_saida = '.png' if extensao.lower() != '.png' else extensao
        nome_arquivo_saida = f"{nome_arquivo_sem_ext}_processado{extensao_saida}"
        caminho_completo_saida = os.path.join(raiz, nome_arquivo_saida)

        try:
            img = Image.open(caminho_completo_saida)
            img = img.convert('RGB')
            imagens.append(img)
            print(f"Imagem lida: {caminho_completo_saida}")
        except Exception as e:
            print(f"Não foi possível ler a imagem {caminho_completo_saida}: {e}")
    return imagens

In [ ]:
# Lendo as Imagens para criação dos Embeddings:
lista = pd.read_csv("database/indices_imagens_preprocessadas.csv")
lista_imagens = lista["caminho_original"]
raiz = "database/natural_images_without_bg/"
todas_as_imagens = ler_imagens(lista_imagens, raiz)
print(f"\nTotal de imagens encontradas: {len(todas_as_imagens)}")

# Você pode acessar as imagens:
# todas_as_imagens[0] # Mostra a primeira imagem

In [ ]:
# Lendo algoritmo ViT - PreTrained
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

In [ ]:
# O processor transforma as imagens para o formato que o modelo espera (pixels, normalização, etc.)
# return_tensors="pt" garante que a saída seja um tensor PyTorch.
inputs = processor(images=todas_as_imagens[0], return_tensors="pt")
# 2. Passar as imagens pré-processadas pelo modelo ViT
with torch.no_grad():
    outputs = model(**inputs)
# Pega o embedding
cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [1, 768]
# Convertendo os logits para numpy (seu vetor de embeddings)
img_embds = cls_embedding.numpy()
# Normalize os embeddings (para usar distância do cosseno)
img_embds = img_embds / np.linalg.norm(img_embds, axis=1, keepdims=True)
# Definindo o índice FAISS para busca vetorial
d = img_embds.shape[1]  # dimensão dos embeddings
# Crie um índice para produto interno (proxy para distância do cosseno)
index = faiss.IndexFlatIP(d)
# Adicione os embeddings normalizados
index.add(img_embds)
print(f"Número de itens no índice: {index.ntotal}")

In [ ]:
# Visualizano o vetor de embeddings
img_embds

In [ ]:
# Visualizano o vetor de embeddings
img_embds.shape

In [ ]:
for i in range(1,len(todas_as_imagens)):
    # O processor transforma as imagens para o formato que o modelo espera (pixels, normalização, etc.)
    # return_tensors="pt" garante que a saída seja um tensor PyTorch.
    inputs = processor(images=todas_as_imagens[i], return_tensors="pt")
    # 2. Passar as imagens pré-processadas pelo modelo ViT
    with torch.no_grad():
        outputs = model(**inputs)
    # Pega o embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [1, 768]
    # Convertendo os logits para numpy (seu vetor de embeddings)
    img_embds = cls_embedding.numpy()

    # Normalize os embeddings (para usar distância do cosseno)
    img_embds = img_embds / np.linalg.norm(img_embds, axis=1, keepdims=True)

    # Adicione os embeddings normalizados
    index.add(img_embds)
    print(f"Número de itens no índice: {index.ntotal}")

In [ ]:
# Suponha que 'index' seja seu índice FAISS
faiss.write_index(index, 'faiss_index_embeddings/vector_databases_faiss_index.index')  # Salva o índice em um arquivo

In [ ]:
# Ler o índice do arquivo
index = faiss.read_index('faiss_index_embeddings/vector_databases_faiss_index.index')  # Carrega o índice de um arquivo

In [ ]:
print(f"Número de itens no índice: {index.ntotal}")